# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# @title Notebook setup.
%cd ..
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import matplotlib.patches as patches
from semantic_routing.benchmark import utils
from semantic_routing.benchmark.graphs import grid_graph
from semantic_routing.benchmark.graphs import city_graph

# Points-of-Interest

Our dataset supports over 200 types of points-of-interest (POIs), largely inspired by OpenStreetMap tags.

## Points-of-interest Specifications

Each type of point-of-interest belongs to either a general category such as "shop" or "amenity", or a specialized category such as "healthcare" or "cuisine".
To get a better understanding of the supported POI types, we can retrieve supported POI specifications by calling `utils.get_poi_specs()`.
These specifications indicate the ID number that is assigned to each POI type, how frequently a POI type is expected to be found, and the name and category of the POI type.


In [2]:
# @title View specifications for POI types belonging to general categories.

poi_specs = utils.get_poi_specs()
general_poi_specs, specialized_poi_specs = poi_specs

poi_type_to_id = {}
id_to_poi_type = {}
general_pois, specialized_pois = poi_specs
for poi_info in general_pois + sum(specialized_pois.values(), []):
  id_to_poi_type[poi_info['poi_type_id']] = poi_info['poi_type_name']
  poi_type_to_id[poi_info['poi_type_name']] = poi_info['poi_type_id']

pd.DataFrame(general_poi_specs)

We can similarly view the specialized point-of-interest types in aggregate.

In [3]:
# @title View specifications of POI types belonging to specialized categories

print("List of specialized categories:", list(specialized_poi_specs.keys()))

pd.DataFrame(sum(list(specialized_poi_specs.values()), []))

## Points-of-interest On Our Road Graphs

We can visualize the points-of-interest on our road graphs. Since even small road graphs are populated with tens of thousands of points-of-interest, in this section we will only visualize a small set of POI types.

In [4]:
chosen_poi_types = ("fuel", "cafe", "library", "restaurant", "convenience")  # @param
chosen_poi_type_ids = [poi_type_to_id[poi_type] for poi_type in chosen_poi_types]

In [5]:
def draw_poi_graph(graph, pos, nx_graph, title):
  plt.figure(figsize=(6, 6))
  nx.draw(
      nx_graph,
      pos,
      edge_color="#dedede",
      node_color="none",
      node_size=10,
      arrows=False,
  )
  colors = ['orange', 'red', 'blue', 'yellow', 'green', 'purple']
  plotted = []
  for u, v in graph.nx_graph.edges():
    for poi_name, poi_types in graph.get_edge_pois(graph.edge_from_internal[(u, v)]):
      if poi_name in plotted:
        continue
      plotted.append(poi_name)
      if not set(poi_types).intersection(set(chosen_poi_type_ids)):
        continue
      poi_coords = (pos[u] + pos[v]) / 2 + (pos[u] - pos[v]) * 0.25 * np.random.uniform(-1, 1)
      for poi_type_id in poi_types:
        if poi_type_id not in chosen_poi_type_ids:
          continue
        circle = patches.Circle(
            poi_coords,
            radius=0.2,
            fc=colors[chosen_poi_type_ids.index(poi_type_id)],
            edgecolor=colors[chosen_poi_type_ids.index(poi_type_id)],
            linewidth=0.2,
        )
        plt.gca().add_patch(circle)

  legend_labels = []
  for i, poi_type in enumerate(chosen_poi_types):
    legend_labels.append(
        patches.Patch(color=colors[i], label=poi_type.capitalize().replace("_", " "))
    )
  plt.legend(
      handles=legend_labels,
      loc="lower center",
      fontsize=10,
      ncol=len(legend_labels),
  )
  plt.axis("off")
  plt.title(title)
  plt.show()

In our dataset's grid graph simulations, points-of-interest are added to the graph randomly. For each POI type, we target a density that is proportional to how frequently that POI type can be found in the real-world (as indicated by the `freq` column of `utils.get_poi_specs()`).

In [6]:
#@title Visualizing the POIs on a simulated grid graph.

num_nodes = 900 # @param
graph = grid_graph.GridGraph(poi_specs, num_nodes, splits=[0.95, 0, 0.05])
pos = {(x, y): np.array([x, y]) for x, y in graph.nx_graph.nodes()}
nx_graph = graph.nx_graph
title = "Example grid road graph simulation with a subset of POIs highlighted."
draw_poi_graph(graph, pos, nx_graph, title)

In our road graphs of major US cities, we obtain points-of-interest data from the OpenStreetMap project. That is, the depicted points-of-interest correspond to real-world businesses, landmarks, and etc.

In [7]:
#@title Visualizing the POIs of a subgraph of Berkeley.

num_nodes = 20000 # @param
graph = city_graph.CityGraph(poi_specs, num_nodes, seed=0, splits=[0.95, 0, 0.05], use_test_city=True)
nx_graph = graph.nx_graph

pos = {}
ego = None
for u, v, data in nx_graph.edges(data=True):
  if ego is None:
    ego = data["u_lat"], data["u_lon"]
  if u not in pos:
    pos[u] = np.array([1000 * (data["u_lat"] - ego[0]), 1000 * (data["u_lon"] - ego[1])])
  if v not in pos:
    pos[v] = np.array([1000 * (data["v_lat"] - ego[0]), 1000 * (data["v_lon"] - ego[1])])

title = "Example road graph from Berkeley with a subset of POIs highlighted."
draw_poi_graph(graph, pos, nx_graph, title)

We can map these points-of-interest to their real-world counterparts by looking at the POI names.

In [8]:
# @title Example of a road segment's annotated POIs.
graph = city_graph.CityGraph(poi_specs, 20000, seed=0, splits=[0.95, 0, 0.05], use_test_city=True)
data = None
for u, v, data in graph.nx_graph.edges(data=True):
  if len([p for p in data["poi_node_names"] if p is not None]) > 2:
    break
assert data is not None
print("Example of a road segment's POIs:", [p for p in data["poi_node_names"] if p is not None])

## Additional Notes on POIs

As we saw previously, a single road segment might have multiple points-of-interest located on it. We also note that a single point-of-interest might be labeled with multiple POI types. Here are some examples.

In [9]:
# @title Examples of POIs belonging to multiple types.
i = 0
for u, v, data in graph.nx_graph.edges(data=True):
  if data["level"] > 1:
    continue
  for name, poi_types in zip(data["poi_node_names"], data['poi_type_ids']):
    if len(poi_types) > 1:
      print("\"{}\" is labeled with the POI types:".format(name), [id_to_poi_type[poi_type] for poi_type in poi_types])
      i += 1
  if i > 10:
    break

In addition, a single point-of-interest might be associated with up to two edges. Note that a POI will only be associated with more than one edge if it is found next to a bidirectional road; in such a case, the POI will be associated with both directions of the road, which are represented in our bidirectional graph as distinct edges.

In [10]:
# @title Example of POI found on two edges.
seen = {}
overlap_found = False
for u, v, data in graph.nx_graph.edges(data=True):
  if data["level"] > 1:
    continue
  for name, poi_types, node_id in zip(data["poi_node_names"], data['poi_type_ids'], data["poi_node_ids"]):
    if node_id not in seen:
      seen[node_id] = (u, v)
    else:
      assert (v, u) == seen[node_id]
      if not overlap_found and name is not None:
        print("The point of interest \"{}\" was found on both edges {} and {}.".format(name, (u[0], v[0]), (seen[node_id][0][0], seen[node_id][1][0])))
        overlap_found = True

# Driving Preferences

It is possible to augment our dataset to specify different driving preferences. As an example, our dataset comes preloaded with support for a preference/aversion towards driving on highways.
To incorporate these preferences when computing ground-truth optimal routes, we assign a penalty/reward for every unit of time spent on a highway.

In [11]:
# @title Realized costs of driving on a highway or residential street under different driving preferences.

print("Driving on a highway for 1 unit of time incurs a realized cost of...")
cost_fn_input = None, None, {"highway": "motorway", "current_travel_time": 10, "highways": ["motorway"], "current_travel_times": [10]}
print("(Neutral)", utils.get_modified_cost_fn("")(*cost_fn_input))
print("(Likes highways)", utils.get_modified_cost_fn("like highways")(*cost_fn_input))
print("(Dislikes highways)", utils.get_modified_cost_fn("dislike highways")(*cost_fn_input))
print()
print("Driving on a residential street for 1 unit of time incurs a realized cost of...")
cost_fn_input = None, None, {"highway": "residential", "current_travel_time": 10, "highways": ["residential"], "current_travel_times": [10]}
print("(Neutral)", utils.get_modified_cost_fn("")(*cost_fn_input))
print("(Likes highways)", utils.get_modified_cost_fn("like highways")(*cost_fn_input))
print("(Dislikes highways)", utils.get_modified_cost_fn("dislike highways")(*cost_fn_input))